In [ ]:
from datasets import Dataset, load_dataset
task_dataset = load_dataset("json", data_files="./datasets/commonsense_170k/train.json")["train"]
new_splits = task_dataset.train_test_split(test_size=160000)
train_dataset, dev_dataset = new_splits["test"], new_splits["train"]
train_dataset.to_json("./datasets/commonsense_160k/train.json")
dev_dataset.to_json("./datasets/commonsense_10k/train.json")

In [ ]:
task_dataset = load_dataset("json", data_files="./datasets/math_10k/train.json")["train"]
new_splits = task_dataset.train_test_split(test_size=9000)
train_dataset, dev_dataset = new_splits["test"], new_splits["train"]
train_dataset.to_json("./datasets/math_9k/train.json")
dev_dataset.to_json("./datasets/math_1k/train.json")